In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# yellowbrick designed to aid in the visualization and interpretation 
# I use it to dimensionality reduction

In [ ]:
!pip install yellowbrick 

In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from yellowbrick.target import FeatureCorrelation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors 
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [ ]:
data = pd.read_csv("C:\\Users\\hoorm\\Desktop\\Codes\\Internship Code Alpha\\CodeAlpha_ MUSIC RECOMMDATION SYSTEM\\data.csv")
genre_data = pd.read_csv("C:\\Users\\hoorm\\Desktop\\Codes\\Internship Code Alpha\\CodeAlpha_ MUSIC RECOMMDATION SYSTEM\\data_by_genres.csv")
year_data = pd.read_csv("C:\\Users\\hoorm\\Desktop\\Codes\\Internship Code Alpha\\CodeAlpha_ MUSIC RECOMMDATION SYSTEM\\data_by_year.csv")

In [ ]:
data

In [ ]:
print(data.info())

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.valence.hist()

In [ ]:
data.year.hist()

In [ ]:
data.acousticness.hist()

In [ ]:
data.danceability.hist()

In [ ]:
data.duration_ms.hist()

In [ ]:
data.energy.hist()

In [ ]:
data.explicit.hist()

In [ ]:
data.instrumentalness.hist()

In [ ]:
data.key.hist()

In [ ]:
data.liveness.hist()

In [ ]:
data.loudness.hist()

In [ ]:
data.popularity.hist()

In [ ]:
data.speechiness.hist()

In [ ]:
data.tempo.hist()

In [ ]:
genre_data

In [ ]:
genre_data.describe()

In [ ]:
print(genre_data.info())

In [ ]:
genre_data.isnull().sum()

In [ ]:
genre_data.shape

In [ ]:
genre_data.acousticness.hist()

In [ ]:
genre_data.danceability.hist()

In [ ]:
genre_data.duration_ms.hist()

In [ ]:
genre_data.energy.hist()

In [ ]:
genre_data.instrumentalness.hist()

In [ ]:
genre_data.liveness.hist()

In [ ]:
genre_data.loudness.hist()

In [ ]:
genre_data.speechiness.hist()

In [ ]:
genre_data.tempo.hist()

In [ ]:
genre_data.valence.hist()

In [ ]:
genre_data.popularity.hist()

In [ ]:
genre_data.key.hist()

In [ ]:
year_data

In [ ]:
year_data.describe()

In [ ]:
print(year_data.info())

In [ ]:
year_data.isnull().sum()

In [ ]:
year_data.shape

In [ ]:
year_data.year.hist()

In [ ]:
year_data.acousticness.hist()

In [ ]:
year_data.danceability.hist()

In [ ]:
year_data.duration_ms.hist()

In [ ]:
year_data.energy.hist()

In [ ]:
year_data.instrumentalness.hist()

In [ ]:
year_data.liveness.hist()

In [ ]:
year_data.loudness.hist()

In [ ]:
year_data.speechiness.hist()

In [ ]:
year_data.tempo.hist()

In [ ]:
year_data.valence.hist()

In [ ]:
year_data.popularity.hist()

In [ ]:
year_data.key.hist()

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration_ms','explicit','key','mode','year']

X, y = data[feature_names], data['popularity']

# Create a list of the feature names
features = np.array(feature_names)

# Instantiate the visualizer
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(20,20)
visualizer.fit(X, y)     
visualizer.show()

In [ ]:
# Compute the correlation matrix
correlation_matrix = X.corr()

# Create a heatmap using Plotly
fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.columns,
    colorscale='Viridis',
    colorbar=dict(title='Correlation')
))

# Update layout for better readability
fig.update_layout(
    title='Feature Correlation Matrix',
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis=dict(tickangle=-45),
    yaxis=dict(tickangle=0),
    autosize=True,
    width=800,
    height=800
)

fig.show()

In [ ]:
# processes a DataFrame containing song years
# Function to get the decade
def get_decade(year):
    period_start = int(year / 10) * 10
    decade = '{}s'.format(period_start)
    return decade

# Assuming 'data' is your DataFrame and it has a 'year' column
data['decade'] = data['year'].apply(get_decade)

# Create a Plotly Express histogram to replicate the Seaborn countplot
fig = px.histogram(data_frame=data, x='decade', 
                   labels={'decade': 'Decade', 'count': 'Count'}, title='Songs by Decade')

fig.update_layout(
    xaxis_title='Decade',
    yaxis_title='Count',
    title_font=dict(size=20),
    xaxis=dict(tickfont=dict(size=12)),
    yaxis=dict(tickfont=dict(size=12)),
    bargap=0.1,  
    bargroupgap=0.1   
)

fig.show()

In [ ]:
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
fig = px.line(year_data, x='year', y=sound_features)
fig.show()

In [ ]:
top10_genres = genre_data.nlargest(10, 'popularity')

fig = px.bar(top10_genres, x='genres', y=['valence', 'energy', 'danceability', 'acousticness'], barmode='group')
fig.show()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Extract feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for the feature importances
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# Sort the features by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)


# Alternatively, plot the feature importances using Plotly
fig = px.bar(importance_df, x='Importance', y='Feature', title='Feature Importance')
fig.show()

In [ ]:
# Group by 'mode' and calculate the mean popularity
fig, ax = plt.subplots(figsize=(8, 5))
data.groupby('mode')['popularity'].mean().plot(kind='bar', ax=ax, fontsize=14, rot='horizontal')

# Set the title and labels
plt.title("Mode and Popularity", fontdict={'fontsize':20})
plt.ylabel('Average Popularity', fontdict={'fontsize':14})
plt.xlabel('')

# Add annotations based on mode values
for index, value in enumerate(data.groupby('mode')['popularity'].mean()):
    if index == 0:
        ax.text(index, value + 0.1, 'Happy', ha='center', va='bottom', fontsize=12, color='blue')
    elif index == 1:
        ax.text(index, value + 0.1, 'Sad', ha='center', va='bottom', fontsize=12, color='red')

plt.show()

In [ ]:
# Corrected Pipeline definition
cluster_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('kmeans', KMeans(n_clusters=10))
])

# Selecting numerical data from genre_data
X = genre_data.select_dtypes(np.number)

# Fitting the pipeline to the data
cluster_pipeline.fit(X)

# Adding cluster labels to the original dataframe
genre_data['cluster'] = cluster_pipeline.predict(X)


In [ ]:
# Visualizing the Clusters with t-SNE

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=1))])
genre_embedding = tsne_pipeline.fit_transform(X) # 2D representation of the data.
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_data['genres']
projection['cluster'] = genre_data['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()

In [ ]:
# Corrected Pipeline definition
song_cluster_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('kmeans', KMeans(n_clusters=20, verbose=False))
], verbose=False)

# Selecting numerical data from the data
X = data.select_dtypes(np.number)
number_cols = list(X.columns)

# Fitting the pipeline to the data
song_cluster_pipeline.fit(X)

# Predicting cluster labels and adding them to the original dataframe
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

In [ ]:
# Visualizing the Clusters with PCA

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'title'])
fig.show()

In [ ]:
#Build Recommender System

In [ ]:
# Inspect the DataFrame to identify correct column names
print(data.head())
print(data.columns)

In [ ]:
# Feature engineering:
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

# If you don't have last listen time, you can't compute 'time_since_last_listen'.
# Instead, you may use available features directly.
data['listen_frequency'] = data.groupby(['id'])['release_date'].transform('count')

In [ ]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Build and train the KNN model
knn = KNeighborsClassifier(n_neighbors=5)  
knn.fit(X_train_scaled, y_train)

# Make predictions
y_pred = knn.predict(X_test_scaled)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
#save the model 
joblib.dump(knn, 'knn_recommender_model.pkl')

In [ ]:
# Read data
songs_data = pd.read_csv("C:\\Users\\hoorm\\Desktop\\Codes\\Internship Code Alpha\\CodeAlpha_ MUSIC RECOMMDATION SYSTEM\\data_by_artist.csv")

# Print column names to find the correct one
print(songs_data.columns)

In [ ]:
# Define a function to map mode values to emotions
def map_mode_to_emotion(mode):
    return 'happy' if mode == 0 else 'sad'

# Apply the function to create the new 'emotion' column
songs_data['emotion'] = songs_data['mode'].apply(map_mode_to_emotion)
songs_data

In [ ]:
# Prepare data
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence']
X = songs_data[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Use Nearest Neighbors to find similar songs
nn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute')
nn.fit(X_scaled)

# Function to recommend similar songs based on the index
def recommend_songs(index, top_n=10):
    if index not in songs_data.index:
        raise ValueError(f"Index {index} not found in the dataset")
    
    distances, indices = nn.kneighbors([X_scaled[index]], n_neighbors=top_n+1)
    similar_indices = indices[0][1:]  # Exclude the song itself
    return songs_data.iloc[similar_indices]

# Recommend songs based on index
index_to_recommend = 20053 
recommended_songs = recommend_songs(index=index_to_recommend)
print(recommended_songs)